In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

builder = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [2]:
# create silver database
spark.sql(
    """
    CREATE DATABASE IF NOT EXISTS PL7G1_GOLD LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/'
    """
)

DataFrame[]

In [2]:
spark.sql(
    """
    DROP TABLE IF EXISTS PL7G1_GOLD.Reciclagem
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE PL7G1_GOLD.Reciclagem (
        Year INT,
        Borough VARCHAR(50),
        SUM_Diversion_Rate_Total Double,
        AVG_Diversion_Rate_Total Double,
        MAX_Diversion_Rate_Total Double,
        MIN_Diversion_Rate_Total Double,
        SUM_Capture_Rate_Paper Double,
        AVG_Capture_Rate_Paper Double,
        MAX_Capture_Rate_Paper Double,
        MIN_Capture_Rate_Paper Double,
        SUM_Capture_Rate_MGP Double,
        AVG_Capture_Rate_MGP Double,
        MAX_Capture_Rate_MGP Double,
        MIN_Capture_Rate_MGP Double,
        SUM_Capture_Rate_Total Double,
        AVG_Capture_Rate_Total Double,
        MAX_Capture_Rate_Total Double,
        MIN_Capture_Rate_Total Double
    )
    USING DELTA
    LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Reciclagem'
    """
)

DataFrame[]

In [3]:
from pyspark.sql.functions import substring, avg, sum, max, min
import pyspark.sql.functions as sf

# read data from the silver tables
reciclagem = spark.table("PL7G1_SILVER.Reciclagem")

reciclagem = reciclagem \
           .groupBy("Year", "Borough") \
           .agg(sum(reciclagem.Diversion_Rate_Total).alias("SUM_Diversion_Rate_Total"),
                avg(reciclagem.Diversion_Rate_Total).alias("AVG_Diversion_Rate_Total"),
                max(reciclagem.Diversion_Rate_Total).alias("MAX_Diversion_Rate_Total"),
                min(reciclagem.Diversion_Rate_Total).alias("MIN_Diversion_Rate_Total"),
                sum(reciclagem.Capture_Rate_Paper).alias("SUM_Capture_Rate_Paper"),
                avg(reciclagem.Capture_Rate_Paper).alias("AVG_Capture_Rate_Paper"),
                max(reciclagem.Capture_Rate_Paper).alias("MAX_Capture_Rate_Paper"),
                min(reciclagem.Capture_Rate_Paper).alias("MIN_Capture_Rate_Paper"),
                sum(reciclagem.Capture_Rate_MGP).alias("SUM_Capture_Rate_MGP"),
                avg(reciclagem.Capture_Rate_MGP).alias("AVG_Capture_Rate_MGP"),
                max(reciclagem.Capture_Rate_MGP).alias("MAX_Capture_Rate_MGP"),
                min(reciclagem.Capture_Rate_MGP).alias("MIN_Capture_Rate_MGP"),
                sum(reciclagem.Capture_Rate_Total).alias("SUM_Capture_Rate_Total"),
                avg(reciclagem.Capture_Rate_Total).alias("AVG_Capture_Rate_Total"),
                max(reciclagem.Capture_Rate_Total).alias("MAX_Capture_Rate_Total"),
                min(reciclagem.Capture_Rate_Total).alias("MIN_Capture_Rate_Total"),
               )

In [4]:
reciclagem.toPandas()

Year        Borough  SUM_Diversion_Rate_Total  AVG_Diversion_Rate_Total  \
0   2017         Queens                    3155.1                 18.780357   
1   2017       Brooklyn                    3532.1                 16.352315   
2   2017      Manhattan                    2942.1                 20.431250   
3   2017          Bronx                    1899.7                 13.192361   
4   2017  Staten Island                     733.5                 20.375000   
5   2016       Brooklyn                    3404.4                 15.761111   
6   2016         Queens                    2998.5                 17.848214   
7   2016      Manhattan                    2886.9                 20.047917   
8   2016  Staten Island                     712.8                 19.800000   
9   2016          Bronx                    1852.1                 12.861806   
10  2018      Manhattan                    2944.2                 20.445833   
11  2018         Queens                    3280.1                 19.524405   
12  2018          Bronx                    2016.1                 14.000694   
13  2018       Brooklyn                    3657.3                 16.931944   
14  2018  Staten Island                     741.3                 20.591667   
15  2019  Staten Island                     751.7                 20.880556   
16  2019       Brooklyn                    3643.5                 16.868056   
17  2019      Manhattan                    3028.3                 21.029861   
18  2019         Queens                    3260.1                 19.405357   
19  2019          Bronx                    2074.3                 14.404861   

    MAX_Diversion_Rate_Total  MIN_Diversion_Rate_Total  \
0                       32.8                      10.9   
1                       33.1                       8.1   
2                       30.1                      11.4   
3                       22.1                       6.3   
4                       23.7                      18.2   
5                       33.8                       7.7   
6                       27.0                      10.4   
7                       29.8                      10.7   
8                       24.7                      17.4   
9                       23.1                       5.9   
10                      28.4                      10.9   
11                      30.0                      11.2   
12                      22.4                       6.4   
13                      31.7                       8.7   
14                      24.6                      17.9   
15                      29.5                      17.9   
16                      31.1                       9.3   
17                      29.8                      10.9   
18                      27.9                      13.0   
19                      23.7                       6.8   

    SUM_Capture_Rate_Paper  AVG_Capture_Rate_Paper  MAX_Capture_Rate_Paper  \
0                   6757.5               40.223214                    65.1   
1                   8594.4               39.788889                    72.6   
2                   5915.2               41.077778                    67.4   
3                   4527.4               31.440278                    49.3   
4                   1614.2               44.838889                    55.1   
5                   8399.4               38.886111                    61.4   
6                   6752.4               40.192857                    65.2   
7                   5889.0               40.895833                    59.2   
8                   1627.3               45.202778                    55.3   
9                   4553.8               31.623611                    49.7   
10                  5818.3               40.404861                    60.4   
11                  6598.2               39.275000                    63.6   
12                  4548.1               31.584028                    47.9   
13                  8574.3       

In [5]:
# write to delta table
reciclagem \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Reciclagem")

In [6]:
spark.table("PL7G1_GOLD.Reciclagem").toPandas()

Year        Borough  SUM_Diversion_Rate_Total  AVG_Diversion_Rate_Total  \
0   2017         Queens                    3155.1                 18.780357   
1   2017       Brooklyn                    3532.1                 16.352315   
2   2017      Manhattan                    2942.1                 20.431250   
3   2017          Bronx                    1899.7                 13.192361   
4   2017  Staten Island                     733.5                 20.375000   
5   2016       Brooklyn                    3404.4                 15.761111   
6   2016         Queens                    2998.5                 17.848214   
7   2016      Manhattan                    2886.9                 20.047917   
8   2016  Staten Island                     712.8                 19.800000   
9   2016          Bronx                    1852.1                 12.861806   
10  2018      Manhattan                    2944.2                 20.445833   
11  2018         Queens                    3280.1                 19.524405   
12  2018          Bronx                    2016.1                 14.000694   
13  2018       Brooklyn                    3657.3                 16.931944   
14  2018  Staten Island                     741.3                 20.591667   
15  2019  Staten Island                     751.7                 20.880556   
16  2019       Brooklyn                    3643.5                 16.868056   
17  2019      Manhattan                    3028.3                 21.029861   
18  2019         Queens                    3260.1                 19.405357   
19  2019          Bronx                    2074.3                 14.404861   

    MAX_Diversion_Rate_Total  MIN_Diversion_Rate_Total  \
0                       32.8                      10.9   
1                       33.1                       8.1   
2                       30.1                      11.4   
3                       22.1                       6.3   
4                       23.7                      18.2   
5                       33.8                       7.7   
6                       27.0                      10.4   
7                       29.8                      10.7   
8                       24.7                      17.4   
9                       23.1                       5.9   
10                      28.4                      10.9   
11                      30.0                      11.2   
12                      22.4                       6.4   
13                      31.7                       8.7   
14                      24.6                      17.9   
15                      29.5                      17.9   
16                      31.1                       9.3   
17                      29.8                      10.9   
18                      27.9                      13.0   
19                      23.7                       6.8   

    SUM_Capture_Rate_Paper  AVG_Capture_Rate_Paper  MAX_Capture_Rate_Paper  \
0                   6757.5               40.223214                    65.1   
1                   8594.4               39.788889                    72.6   
2                   5915.2               41.077778                    67.4   
3                   4527.4               31.440278                    49.3   
4                   1614.2               44.838889                    55.1   
5                   8399.4               38.886111                    61.4   
6                   6752.4               40.192857                    65.2   
7                   5889.0               40.895833                    59.2   
8                   1627.3               45.202778                    55.3   
9                   4553.8               31.623611                    49.7   
10                  5818.3               40.404861                    60.4   
11                  6598.2               39.275000                    63.6   
12                  4548.1               31.584028                    47.9   
13                  8574.3       

In [7]:
spark.sql("""
GENERATE symlink_format_manifest FOR TABLE delta.`hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Reciclagem/`
""").show()

++
||
++
++



In [8]:
spark.sql("""
DROP TABLE IF EXISTS PL7G1_GOLD.Reciclagem_presto
""").show()

spark.sql("""
CREATE EXTERNAL TABLE IF NOT EXISTS PL7G1_GOLD.Reciclagem_presto (
        Year INT,
        Borough VARCHAR(50),
        SUM_Diversion_Rate_Total Double,
        AVG_Diversion_Rate_Total Double,
        MAX_Diversion_Rate_Total Double,
        MIN_Diversion_Rate_Total Double,
        SUM_Capture_Rate_Paper Double,
        AVG_Capture_Rate_Paper Double,
        MAX_Capture_Rate_Paper Double,
        MIN_Capture_Rate_Paper Double,
        SUM_Capture_Rate_MGP Double,
        AVG_Capture_Rate_MGP Double,
        MAX_Capture_Rate_MGP Double,
        MIN_Capture_Rate_MGP Double,
        SUM_Capture_Rate_Total Double,
        AVG_Capture_Rate_Total Double,
        MAX_Capture_Rate_Total Double,
        MIN_Capture_Rate_Total Double
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
STORED AS INPUTFORMAT 'org.apache.hadoop.hive.ql.io.SymlinkTextInputFormat'
OUTPUTFORMAT 'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION 'hdfs://hdfs-nn:9000/Poluicao_ar/Gold/PL7G1_GOLD.db/Reciclagem/_symlink_format_manifest/'
""").show()

++
||
++
++

++
||
++
++

